In [174]:
import pandas as pd
import numpy as np

In [175]:
from gp_pref_elicit_luisa import gaussian_process as GP 
from gp_pref_elicit_luisa import dataset as data 
from gp_pref_elicit_luisa import acquisition_function as acquisition_function
from gp_pref_elicit_luisa.gp_utilities import utils_ccs as gp_utils_ccs
from gp_pref_elicit_luisa.gp_utilities import utils_data as gp_utils_data
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users

In [176]:
# gp_utils_ccs.get_ccs(2, 20)
# outputs a synthetic Pareto Coverage Set of value vectors with 2 objectives and 20 datapoints

The original dataset, in the data_preprocessed.ipynb file, can not be used for the Gaussian Process as it is not in the form of value vectors. To bring it into the form of value vectors, the dataset needs to be passed into a Multi-Objective Shortest Path Planning Solver which will output value vectors. These value vectors can then be an input for the GP. Thus, for now, I am using a synthetic Pareto Coverage Set (PCS) with 2 objectives and 20 datapoints which gives us synthetic value vectors for input to the GP.

In [177]:
synthetic_pcs = np.array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])
synthetic_pcs 

array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])

In [178]:
# initializing GP, Dataset and Acquisition Function 
# num_objectives = 2
GP = GP.GPPairwise(2)
utils_comparisons = data.DatasetPairwise(2)
acquisition_function_DA = acquisition_function.DiscreteAcquirer(input_domain=synthetic_pcs, query_type='pairwise', seed=None, acquisition_type='expected improvement')
acquisition_function_EI = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)

In [179]:
# getting the user preferences for generating a ground truth utility function
user_pref = gp_utils_users.UserPreference(2, std_noise=0.1)

In [180]:
# getting the datapoints at which we need to query the user first 
start_points = acquisition_function_DA.get_start_points(gaussian_process=GP)
start_points


(array([0.14370116, 0.99159928]), array([0.99525346, 0.08213535]))

In [152]:
# generating utilities for the starting datapoints
ground_truth_utility_func = user_pref.get_preference(start_points, add_noise=True)
ground_truth_utility_func

array([0.25451411, 0.7465639 ])

In [153]:
# ground_truth_utility_func = user_pref.get_preference(synthetic_pcs, add_noise=True)
# ground_truth_utility_func

In [154]:
highest_utility_index = np.argmax(ground_truth_utility_func)
lowest_utility_index = np.argmin(ground_truth_utility_func)
print('Highest utility is at index ', highest_utility_index,  'for the user in the dataset', '\n', 
'Lowest utility is at index ', lowest_utility_index, 'for the user in the dataset')
# this outputs the index of the points in the synthetic dataset which have the highest and lowest utility

Highest utility is at index  1 for the user in the dataset 
 Lowest utility is at index  0 for the user in the dataset


In [155]:
# getting the user preference for the datapoint in the synthetic dataset having the highest utility
# ground_truth_utility_func = user_pref.get_preference(synthetic_pcs[highest_utility_index], add_noise=True)
# ground_truth_utility_func

In [156]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons.add_single_comparison(synthetic_pcs[highest_utility_index], synthetic_pcs[lowest_utility_index])
utils_comparisons_datapoints_start = utils_comparisons.datapoints
utils_comparisons_datapoints_start

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928]])

In [157]:
# to get the point which will be retained by the comparison in the dataset
highest_point = np.argmax(utils_comparisons_datapoints_start)
highest_point

3

In [158]:
# updating the GP based on the comparisons added to the dataset
GP.update(utils_comparisons)

In [159]:
# gives the next points to query for the user 
next_points = acquisition_function_DA.get_next_point(gaussian_process=GP, dataset=utils_comparisons)
next_points
# outputs next set of value vectors for comparisons

array([0.81501748, 0.65358114])

In [160]:
# we now run the user preference on the next points to query
ground_truth_utility_func = user_pref.get_preference(x=next_points, add_noise=True)
ground_truth_utility_func

array([0.67667458])

In [161]:
# we get the index of the highes point which indicates highest utility for the datapoint
highest_utility_index1 = np.argmax(ground_truth_utility_func)
highest_utility_index1

0

In [162]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons_next = utils_comparisons.add_single_comparison(synthetic_pcs[highest_point], synthetic_pcs[highest_utility_index1])
utils_comparisons_datapoints_next = utils_comparisons.datapoints
utils_comparisons_datapoints_next 

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928],
       [0.91055917, 0.45020785]])

In [163]:
GP.update(utils_comparisons)

In [164]:
# points to exclude for getting the expected improvement
# we do this because we want to exclude all the points the current maximum was already compared to
exclude = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)
exclude

array([[0.91055917, 0.45020785],
       [0.14370116, 0.99159928]])

In [165]:
next_point_EI = acquisition_function_DA.get_next_point_EI(gaussian_process=GP, exclude=exclude)
next_point_EI

array([0.59678925, 0.81854996])

In [166]:
# generating utilities for the next datapoints according to EI
ground_truth_utility_func1 = user_pref.get_preference(next_point_EI, add_noise=True)
ground_truth_utility_func1

array([0.58425793])

In [167]:
# we get the index of the highes point which indicates highest utility for the datapoint
highest_utility_index2 = np.argmax(ground_truth_utility_func1)
highest_utility_index2

0

In [168]:
# comparing datapoints with highest and lowest utility values generated from the ground truth utility function 
# and adding these to the GP
utils_comparisons.add_single_comparison(synthetic_pcs[highest_point], synthetic_pcs[highest_utility_index1])
utils_comparisons_datapoints = utils_comparisons.datapoints
utils_comparisons_datapoints 

array([[0.9797389 , 0.2242916 ],
       [0.14370116, 0.99159928],
       [0.91055917, 0.45020785]])

In [169]:
GP.update(utils_comparisons)

In [170]:
# similarly for this, I am not sure if the input and output are correct
acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)

array([0.08832413, 0.12961103, 0.17986479, 0.12961103, 0.24691373,
       0.23225387, 0.18015988, 0.23207685, 0.20731343, 0.21313976,
       0.1830873 , 0.22240784, 0.1455025 , 0.10007696, 0.13540019,
       0.14193131, 0.24577038, 0.11965747, 0.24273999, 0.188506  ])

In [171]:
prob_impr = acquisition_function.get_probability_of_improvement(x=next_point_EI, gaussian_process=GP, x_previous=next_points)
prob_impr

array([0.49035336])

In [182]:
prob_impr = 0.5
while prob_impr > 0.1:
    start_points = acquisition_function_DA.get_start_points(gaussian_process=GP)
    ground_truth_utility_function_start = user_pref.get_preference(start_points, add_noise=True)
    highest_utility_index_start = np.argmax(ground_truth_utility_function_start)
    lowest_utility_index_start = np.argmin(ground_truth_utility_function_start)
    utils_comparisons.add_single_comparison(synthetic_pcs[highest_utility_index_start], synthetic_pcs[lowest_utility_index_start])
    utility_comparisons_datapoints_start = utils_comparisons.datapoints
    highest_utility_comparisons_datapoints_start = np.argmax(utility_comparisons_datapoints_start)
    GP.update(utils_comparisons)

    next_points = acquisition_function_DA.get_next_point(gaussian_process=GP, dataset=utils_comparisons)
    ground_truth_utility_function_next = user_pref.get_preference(x=next_points, add_noise=True)
    highest_utility_index_next = np.argmax(ground_truth_utility_function_next)
    utils_comparisons.add_single_comparison(synthetic_pcs[highest_utility_comparisons_datapoints_start], synthetic_pcs[highest_utility_index_next])
    utility_comparisons_datapoints_next = utils_comparisons.datapoints
    highest_utility_comparisons_datapoints_next = np.argmax(utility_comparisons_datapoints_next)
    GP.update(utils_comparisons)

    exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)

    next_point_EI = acquisition_function_DA.get_next_point_EI(gaussian_process=GP, exclude=exclude_points)
    ground_truth_utility_function_next_EI = user_pref.get_preference(next_point_EI, add_noise=True)
    highest_utility_index_next_EI = np.argmax(ground_truth_utility_function_next_EI)
    utils_comparisons.add_single_comparison(synthetic_pcs[highest_utility_comparisons_datapoints_next], synthetic_pcs[highest_utility_index_next_EI])
    utility_comparisons_datapoint_next_EI = utils_comparisons.datapoints
    highest_utility_comparisons_datapoints_next_EI = np.argmax(utility_comparisons_datapoint_next_EI)
    GP.update(utils_comparisons)

    expected_impr_points = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)
    prob_impr = acquisition_function.get_probability_of_improvement(x=next_point_EI, gaussian_process=GP, x_previous=next_points)
    print('start points for the GP: ', start_points, '\n', 
          'Ground truth utility for starting points: ', ground_truth_utility_function_start, '\n',
          'Highest utility index for start points: ', highest_utility_index_start, '\n',
          'Lowest utility index for starting points: ', lowest_utility_index_start, '\n',

          'Next points for the GP: ', next_points, '\n',
          'Ground truth utility function for next points: ', ground_truth_utility_function_next, '\n'
          'Highest utility index for next points: ', highest_utility_index_next, '\n',

          'Excluded points: ', exclude_points, '\n',

          'next points for the GP acc to EI: ', next_point_EI, '\n',
          'Ground truth utility function for the next points acc to EI: ', ground_truth_utility_function_next_EI, '\n',
          'Highest utility index for next point acc to EI: ', highest_utility_index_next_EI, '\n',

          'expected improvement points: ', expected_impr_points, '\n',
          'Probability of improvement: ', prob_impr)
    if prob_impr < 0.05:
        break

start points for the GP:  (array([0.59678925, 0.81854996]), array([0.21487083, 0.97724899])) 
 Ground truth utility for starting points:  [0.60458522 0.45438732] 
 Highest utility index for start points:  0 
 Lowest utility index for starting points:  1 
 Next points for the GP:  [0.59678925 0.81854996] 
 Ground truth utility function for next points:  [0.59020008] 
Highest utility index for next points:  0 
 Excluded points:  [[0.9797389  0.2242916 ]
 [0.14370116 0.99159928]
 [0.14370116 0.99159928]] 
 next points for the GP acc to EI:  [0.74002719 0.7144284 ] 
 Ground truth utility function for the next points acc to EI:  [0.52254552] 
 Highest utility index for next point acc to EI:  0 
 expected improvement points:  [0.12358125 0.12358125 0.19987442 0.23974792 0.25035059 0.23704558
 0.19302516 0.25034371 0.21578453 0.22653687 0.2024943  0.22870087
 0.15762474 0.1335567  0.1632167  0.2487663  0.25035402 0.15005375
 0.24837059 0.19926164] 
 Probability of improvement:  [0.49376678]
s

KeyboardInterrupt: 

In [29]:
# I haven't updated this for loop according to the code above. Once I know the code above is correct, I will put everything here
num_iter = 2
for i in range(num_iter):
    # sampling on synthetic pcs according to gaussian process
    sampled_points_GP = GP.sample(synthetic_pcs)
    # output of this sampling is the utility values

    # Getting highest and lowest utility values from the sampled points
    highest_util = max(sampled_points_GP)
    lowest_util = min(sampled_points_GP)

    # performing single comparisons on the highest and lowest utility values
    
print('Sampled points from the GP are: \n', sampled_points_GP, '\n' 
    'Highest utility value is: ', highest_util, '\n'
    'Lowest utility value is: ', lowest_util, '\n')

Sampled points from the GP are: 
 [-0.07328946 -0.10317786 -0.21344314  0.57459712  0.0934856  -1.14374158
  0.13320398 -0.02408565 -1.14214467  0.00606222  0.07539892 -1.20085411
 -0.55222648 -0.04297325 -0.1119149   0.65180377 -0.53481497  0.00167548
 -0.19310516 -0.99033449] 
Highest utility value is:  0.6518037744716059 
Lowest utility value is:  -1.200854112518174 

